# 1.1 Prompt Engigeering

System prompts are the easiest way to modify an agent's performance and persona.  

• Role Setting: Using a system prompt can change the agent's behavior (e.g., making it act as a "science fiction writer" to answer questions it normally couldn't).  
• Few-Shot Prompting: You can provide input-output pair examples within the prompt to force the agent to follow a specific style or length.  
• Structural Constraints: Explicitly defining the structure (e.g., asking for four specific topics) prevents the LLM from hitting "self-determined word counts" and keeps answers high-quality.   

Let's see the different techniques in the following examples.

## 1.1.0 Getting Started

Load the required environment variables:

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 1.1.1 Basic Prompting

In [4]:
import os
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)
agent = create_agent(model=model)

question = HumanMessage(content="What's the capital of the moon?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

The Moon doesn’t have a capital, as it is not a sovereign entity or inhabited by any official population. It is a natural satellite of Earth without organized governments, cities, or nations. However, scientific research stations like those proposed for future lunar missions might become the closest equivalent to settlements on the Moon!


In [5]:
system_prompt = "You are a science fiction writer, create a capital city at the users request."

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

The capital city of the Moon is **Lunaris Prime**, a thriving hub of innovation and extraterrestrial collaboration nestled within the sprawling Tycho Crater. Shielded by a cutting-edge, transparent dome made from lunar-regolith-based alloy, Lunaris Prime is a dazzling metropolis that radiates with the glow of solar energy and advanced technology.

The city is divided into distinct districts, including:

1. **The Zenith Spire**: A towering structure at the city's center that serves as both a governmental hub and the Moon's primary communications relay with Earth and deeper space colonies.
2. **Eterna Gardens**: A vast, bioluminescent agricultural sector where oxygen is generated, and fresh food for lunar inhabitants is grown through advanced hydroponics and vertical farming systems.
3. **Cosmica Plaza**: The cultural heart of Lunaris Prime, filled with restaurants, theaters, and museums that explore humanity's journey to the stars and the Moon's role as a stepping stone into the cosmos.

## 1.1.2 Few-shot examples

In [7]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

The capital of the Moon is **Lunaris**. Nestled within the vast expanse of the Sea of Tranquility, Lunaris is a sprawling city built beneath a network of transparent domes. Powered by fusion energy and surrounded by glimmering solar arrays, Lunaris is a beacon of science, diplomacy, and lunar exploration. Its central hub, the Celestial Conclave, serves as the governing seat where representatives from Earth and other celestial colonies convene to shape the future of humanity among the stars.


## 1.1.3 Structured prompts

In [8]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-3 words to describe its vibe

Economy: Main industries

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

**Name:** Lunalith  

**Location:** Sea of Tranquility, near the Apollo 11 landing site  

**Vibe:** Futuristic, serene, luminous  

**Economy:** Helium-3 mining, space tourism, scientific research, interstellar trade hub  


## 1.1.4 Structured output

For more advanced use cases, you can move beyond text blocks by providing an output schema. This forces the model to fill out specific fields (like a form).  
Structured output is vital if you intend to use the agent's response as data for functions down the line, allowing you to pull specific values programmatically.

In [9]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pydantic import BaseModel

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
    response_format=CapitalInfo
)

question = HumanMessage(content="What is the capital of The Moon?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

CapitalInfo(name='Lunaris', location='Mare Imbrium, The Moon', vibe='Futuristic and luminous, characterized by advanced technology and interconnected lunar habitats.', economy='Thrives on space exploration technology exports, scientific research, and interplanetary trade.')

In [10]:
response["structured_response"].name

'Lunaris'

In [11]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name} is a city located at {capital_location}")

Lunaris is a city located at Mare Imbrium, The Moon
